In [15]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jan  2 15:14:19 2019

@author: eiahb
"""
import gensim,os,sklearn,tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from collections import Counter
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from datetime import datetime

from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora
from gensim import models

%matplotlib inline


C:\Users\Evan\AppData\Local\conda\conda\envs\env_futures\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
START_DATE="2018-01-01"
ROOT_DIR_OF_PROJECT="C:\\Users\\Evan\\MyFile\\Fortune-street\\007 oil_price\\oilprice_news_analyze\\"
#ROOT="D:/work/fortune_street/002 news_analyze/002 data/002 corpus_data/"

root_dir=ROOT_DIR_OF_PROJECT+"002 data/002 corpus_data/"
list_of_source=os.listdir(root_dir+"raw_csv_data/")
list_of_source



['cnbc_news_data',
 'oil_price_news_data',
 'uk_reuters_news_data',
 'waiting_preprocess']

In [8]:
class Preprocessor:
    def __init__(self,stem=False):
        self.stem=stem
        
    def stem_and_other_stuff(self,each_news,do_tokenize=False):
        if do_tokenize:
            each_news=word_tokenize(each_news)
        ps=PorterStemmer()
        return([ps.stem(word.lower()) for word in each_news if word.isalpha()])
        
    def check_alpha_tolower(self,each_news,do_tokenize=False):
        if do_tokenize:
            each_news=word_tokenize(each_news)
        return([word.lower() for word in each_news if word.isalpha()])
    
    def my_word_tokenize(self,content):
        try:
            return(word_tokenize(content))
        except:
            print("tokenize fail")
            return([])
    def get_tokens(self,content):
        token_content=content.apply(self.my_word_tokenize)
        if self.stem:        
            tokens=token_content.apply(self.stem_and_other_stuff)
        else:
            tokens=token_content.apply(self.check_alpha_tolower)
        return(tokens)
        
    def get_counter(self,tokens):
        content_counter = Counter()
        for news in tokens:
            content_counter.update(news)
        return(content_counter)
preprocessor=Preprocessor(stem=True)

In [11]:
counter = Counter()
for aSource in list_of_source:
    list_of_filenames=os.listdir(root_dir+"raw_csv_data/"+aSource) 
    for aFile in list_of_filenames:
        filedir=root_dir+"raw_csv_data/"+aSource+"/"+aFile
        news_data_df=pd.read_csv(root_dir+"raw_csv_data/"+aSource+"/"+aFile)
        try:
            news_contents=news_data_df['content']
            news_titles=news_data_df['title']
            news_datetime=news_data_df['publish_datetime']
        except:
            continue
        news_tokens=preprocessor.get_tokens(news_contents)
        news_counter=preprocessor.get_counter(news_tokens)
        counter.update(news_counter)
        
        savedir=root_dir+"token_stemmed_txt_data/"+aSource+"/"+news_datetime[0]+"/"
        if not os.path.exists(savedir):
            os.makedirs(savedir)
        for a_news_id,a_news in enumerate(zip(news_titles,news_tokens)):
            a_title,a_tokens=a_news
            with open(savedir+"news_"+str(a_news_id)+".txt", "w",encoding="utf8") as text_file:
                text_file.writelines(a_title.lower()+"\n")
                for token in a_tokens:
                    text_file.write(token+"\n")
#         os.remove(filedir) 
        print(filedir,"preprocess done!")
                    
dictionary = counter
datetimestr=datetime.today().date().strftime("%Y_%m_%d")
# np.save('D:/work/fortune_street/002 news_analyze/002 data/003 outcome_data/word_counter_data/word_counter_'+datetimestr+'.npy', dictionary) 



C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/cnbc_news_data/cnbc_news_2018_12_04.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/cnbc_news_data/cnbc_news_2018_12_05.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/cnbc_news_data/cnbc_news_2018_12_06.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/cnbc_news_data/cnbc_news_2018_12_07.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/cnbc_news_data/cnbc_news_2018_12_08.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/cnbc_news_data/cnbc_news_2018_12_09.csv preprocess done

C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/cnbc_news_data/cnbc_news_2019_01_29.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/cnbc_news_data/cnbc_news_2019_01_30.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/cnbc_news_data/cnbc_news_2019_01_31.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/cnbc_news_data/cnbc_news_2019_02_01.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/cnbc_news_data/cnbc_news_2019_02_02.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/cnbc_news_data/cnbc_news_2019_02_03.csv preprocess done

C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/oil_price_news_data/oilprice_news_2018_02_09.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/oil_price_news_data/oilprice_news_2018_02_12.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/oil_price_news_data/oilprice_news_2018_02_13.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/oil_price_news_data/oilprice_news_2018_02_14.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/oil_price_news_data/oilprice_news_2018_02_15.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/oil_price_

C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/oil_price_news_data/oilprice_news_2018_04_26.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/oil_price_news_data/oilprice_news_2018_04_27.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/oil_price_news_data/oilprice_news_2018_04_30.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/oil_price_news_data/oilprice_news_2018_05_02.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/oil_price_news_data/oilprice_news_2018_05_03.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/oil_price_

C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/oil_price_news_data/oilprice_news_2018_07_05.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/oil_price_news_data/oilprice_news_2018_07_06.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/oil_price_news_data/oilprice_news_2018_07_09.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/oil_price_news_data/oilprice_news_2018_07_10.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/oil_price_news_data/oilprice_news_2018_07_11.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/oil_price_

...

C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/uk_reuters_news_data/uk_reuters_news_2018_09_05.csv preprocess done!
tokenize fail
tokenize fail
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/uk_reuters_news_data/uk_reuters_news_2018_09_06.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/uk_reuters_news_data/uk_reuters_news_2018_09_07.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/uk_reuters_news_data/uk_reuters_news_2018_09_08.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/uk_reuters_news_data/uk_reuters_news_2018_09_09.csv preprocess done!
tokenize fail
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news

tokenize fail
tokenize fail
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/uk_reuters_news_data/uk_reuters_news_2018_10_18.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/uk_reuters_news_data/uk_reuters_news_2018_10_19.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/uk_reuters_news_data/uk_reuters_news_2018_10_20.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/uk_reuters_news_data/uk_reuters_news_2018_10_21.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/uk_reuters_news_data/uk_reuters_news_2018_10_22.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 d

C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/uk_reuters_news_data/uk_reuters_news_2018_12_02.csv preprocess done!
tokenize fail
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/uk_reuters_news_data/uk_reuters_news_2018_12_03.csv preprocess done!
tokenize fail
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/uk_reuters_news_data/uk_reuters_news_2018_12_04.csv preprocess done!
tokenize fail
tokenize fail
tokenize fail
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/uk_reuters_news_data/uk_reuters_news_2018_12_05.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/uk_reuters_news_data/uk_reuters_news_2018_12_06.csv preprocess done!
tokenize fail
tokenize fail
C:\User

tokenize fail
tokenize fail
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/uk_reuters_news_data/uk_reuters_news_2019_01_16.csv preprocess done!
tokenize fail
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/uk_reuters_news_data/uk_reuters_news_2019_01_17.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/uk_reuters_news_data/uk_reuters_news_2019_01_18.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/uk_reuters_news_data/uk_reuters_news_2019_01_19.csv preprocess done!
tokenize fail
tokenize fail
C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data/002 corpus_data/raw_csv_data/uk_reuters_news_data/uk_reuters_news_2019_01_20.csv preprocess done!
C:\Users\Evan\MyFile\Fortune-street

OSError: Initializing from file failed

# try gensim

In [17]:
def read_news_txt_file(aFilepath):
    token_list=[]
    with open(aFilepath,"r",encoding='utf8') as f:
        news_title=f.readline()
        line = news_title
        while line :
            line = f.readline()
            token_list.append(line.strip())
    return(news_title,token_list)


In [20]:
counter_dir="C:/Users/Evan/MyFile/Fortune-street/007 oil_price/oilprice_news_analyze/002 data/003 outcome_data/word_counter_data/"
txt_dir="C:/Users/Evan/MyFile/Fortune-street/007 oil_price/oilprice_news_analyze/002 data/002 corpus_data/token_stemmed_txt_data/"
# C:\Users\Evan\MyFile\Fortune-street\007 oil_price\oilprice_news_analyze\002 data\002 corpus_data\token_txt_data

list_counter_filenames=list(os.walk(counter_dir))[0][2]
print(list_counter_filenames)
list_txt_source=list(os.walk(txt_dir))[0][1]
print(list_txt_source)

full_list=[]
for aSource in list_txt_source:
    for subpath , null , list_news_filenames in tqdm.tqdm(list(os.walk(txt_dir+aSource))[1:]):
        news_publish_date=subpath.split("\\")[1]
        source=subpath.split("\\")[0].split("/")[-1]
        for aFilename in list_news_filenames:
            aFilepath="/".join([subpath, aFilename])
            news_title,news_tokens_list=(read_news_txt_file(aFilepath))
            
            out_dict={'news_publish_date':news_publish_date,
                      'news_title':news_title,
                      'news_tokens_list':news_tokens_list,
                      'source':source}
            full_list.append(out_dict)

['word_counter.npy', 'word_counter_2019_02_08.npy', 'word_counter_2019_02_12.npy', 'word_counter_2019_02_26.npy']
['cnbc_news_data', 'oil_price_news_data', 'uk_reuters_news_data']



 21%|██▏       | 80/373 [00:46<03:39,  1.33it/s]


 68%|██████▊   | 255/373 [02:15<01:06,  1.77it/s]


100%|██████████| 373/373 [03:21<00:00,  1.32it/s]

In [21]:
full_news_df=pd.DataFrame(full_list)
full_news_df.head(10)

,news_publish_date,news_title,news_tokens_list,source
0,2018-12-04,update 3-oil prices skid with global stock mar...,"[singapor, dec, reuter, oil, price, slump, by,...",cnbc_news_data
1,2018-12-04,u.s. oil prices slide with global markets on e...,"[singapor, dec, reuter, oil, price, slip, by, ...",cnbc_news_data
2,2018-12-04,heating oil futures turn negative in post-sett...,"[heat, oil, futur, turn, neg, in, trade, after...",cnbc_news_data
3,2018-12-04,oil futures fall in post-settlement trade afte...,"[oil, futur, fall, in, trade, after, api, data...",cnbc_news_data
4,2018-12-05,update 1-oil prices dip as stock markets slide...,"[opec, meet, singapor, dec, reuter, oil, price...",cnbc_news_data
5,2018-12-05,oil prices ease in cautious trading ahead of o...,"[singapor, dec, reuter, oil, price, dip, on, t...",cnbc_news_data
6,2018-12-05,"oil prices extend gains, brent crude up by mor...","[oil, price, extend, gain, brent, crude, up, b...",cnbc_news_data
7,2018-12-05,update 5-oil prices fall on global growth conc...,"[singapor, dec, reuter, oil, price, fell, on, ...",cnbc_news_data
8,2018-12-05,update 4-oil prices slip on global growth conc...,"[singapor, dec, reuter, oil, price, fell, on, ...",cnbc_news_data
9,2018-12-06,"oil firms on drop in us crude stocks, market a...","[decis, singapor, dec, reuter, oil, price, sta...",cnbc_news_data


In [22]:
def stop_words_filter(aListOfTokens,stopwords=None):
    new_l=[token for token in aListOfTokens if token not in stopwords]
    return(new_l)

In [23]:
corpus_for_vectorizer=full_news_df.news_tokens_list.apply(lambda x: " ".join(x))
vectorizer=TfidfVectorizer(max_df=0.7,min_df=15)
vectorizer.fit(corpus_for_vectorizer)
defined_stopwords=vectorizer.stop_words_

In [24]:
datetimestr=str(datetime.today().date())
np.save('C:\\Users\\Evan\\MyFile\\Fortune-street\\007 oil_price\\oilprice_news_analyze\\002 data\\003 outcome_data\\stop_words/stop_words_'+datetimestr+'.npy', defined_stopwords) 
full_news_df['news_tokens_list_stripped']=full_news_df.news_tokens_list.apply(stop_words_filter,stopwords=defined_stopwords)
full_news_df

,news_publish_date,news_title,news_tokens_list,source,news_tokens_list_stripped
0,2018-12-04,update 3-oil prices skid with global stock mar...,"[singapor, dec, reuter, oil, price, slump, by,...",cnbc_news_data,"[singapor, dec, oil, price, slump, around, per..."
1,2018-12-04,u.s. oil prices slide with global markets on e...,"[singapor, dec, reuter, oil, price, slip, by, ...",cnbc_news_data,"[singapor, dec, oil, price, slip, percent, ear..."
2,2018-12-04,heating oil futures turn negative in post-sett...,"[heat, oil, futur, turn, neg, in, trade, after...",cnbc_news_data,"[heat, oil, futur, turn, neg, trade, after, ap..."
3,2018-12-04,oil futures fall in post-settlement trade afte...,"[oil, futur, fall, in, trade, after, api, data...",cnbc_news_data,"[oil, futur, fall, trade, after, api, data, sh..."
4,2018-12-05,update 1-oil prices dip as stock markets slide...,"[opec, meet, singapor, dec, reuter, oil, price...",cnbc_news_data,"[opec, meet, singapor, dec, oil, price, fell, ..."
5,2018-12-05,oil prices ease in cautious trading ahead of o...,"[singapor, dec, reuter, oil, price, dip, on, t...",cnbc_news_data,"[singapor, dec, oil, price, dip, thursday, tep..."
6,2018-12-05,"oil prices extend gains, brent crude up by mor...","[oil, price, extend, gain, brent, crude, up, b...",cnbc_news_data,"[oil, price, extend, gain, brent, crude, up, m..."
7,2018-12-05,update 5-oil prices fall on global growth conc...,"[singapor, dec, reuter, oil, price, fell, on, ...",cnbc_news_data,"[singapor, dec, oil, price, fell, wednesday, p..."
8,2018-12-05,update 4-oil prices slip on global growth conc...,"[singapor, dec, reuter, oil, price, fell, on, ...",cnbc_news_data,"[singapor, dec, oil, price, fell, wednesday, p..."
9,2018-12-06,"oil firms on drop in us crude stocks, market a...","[decis, singapor, dec, reuter, oil, price, sta...",cnbc_news_data,"[decis, singapor, dec, oil, price, stabilis, f..."


In [ ]:
DICT_FOLDER = "D:/work/fortune_street/002 news_analyze/002 data/003 outcome_data/gemsim_outcome/dictionary/"
CORPUS_FOLDER="D:/work/fortune_street/002 news_analyze/002 data/003 outcome_data/gemsim_outcome/corpus/"
saving_date=datetime.today().date()
print('Folder "{}" will be used to save dictionary and corpus for {}'.format(DICT_FOLDER,saving_date))
if not os.path.exists(CORPUS_FOLDER):
    os.makedirs(CORPUS_FOLDER)
if not os.path.exists(DICT_FOLDER):
    os.makedirs(DICT_FOLDER)